In [7]:
# Import libraries
import random
import os
import numpy as np 
import pandas as pd 
import requests
import pandas_datareader as web

# Date
import datetime as dt
from datetime import date, timedelta, datetime

# EDA
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

# FE
from tsfresh import extract_features, select_features, extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute
from sklearn.inspection import permutation_importance
import eli5
from eli5.sklearn import PermutationImportance
import shap

# Time Series - EDA and Modelling
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA

# Metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

# Modeling and preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor
from sklearn.neural_network import MLPRegressor
from prophet import Prophet
import xgboost as xgb
from xgboost import XGBRegressor
import lightgbm as lgb
from lightgbm import LGBMRegressor
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

In [8]:
# What EDA & FE techniques use?
is_anomalies = True # or False - Take into account anomalies or no?

In [9]:
# What type of model to use?
is_Prophet = True   # or False - Facebook Prophet
is_ARIMA = False     # or False - ARIMA and AutoARIMA
is_other_ML = True  # or False - multi-factors models: trees, neural networks, etc.

In [10]:
# Automatic building ARIMA for Time Series
if is_ARIMA:
    !pip install pmdarima
    import pmdarima as pm

In [11]:
# Set random state
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

random_state = 42
fix_all_seeds(random_state)

In [12]:
# Set the main parameter
forecasting_days = 7  # forecasting_days > 1

In [13]:
pd.set_option('max_columns',1000)

OptionError: 'Pattern matched multiple keys'

In [14]:
# Set indicator names (see dataset https://www.kaggle.com/datasets/vbmokin/wq-southern-bug-river-01052021)
# all_indicator_names as str : 'NH4', 'BSK5', 'NO3', 'NO2', 'SO4', 'PO4', 'CL'
target_indicator_name = 'NO3'
feature_indicator_names = ['NH4', 'NO2', 'CL']  # if it is necessary to forecast the data, 
                                          # taking into account the data of other indicators

In [ ]:
# Set id of stations
id_target_station = 14
# all_id_station as int: 1-21 (21 - river outlet, 1 - river mouth, 
#                        stations numbering - against the flow of the river)
id_feature_station = [15, 16]  # if it is necessary to forecast the data, 
                               # taking into account the data of other stations

In [ ]:
def get_water_data(target_indicator_name : str, 
                   id_target_station : int,  
                   date_start : str = "2000-01-02", 
                   feature_indicator_names : list = [], # list of str
                   id_feature_station : list = [],      # list of int
                   date_end : str = "2021-06-04"):
    # Get data on the water quality of the Southern Booh (or Bug) River 
    # for the given indicators (target and others, if necessary) 
    # at the given stations (target and others, if necessary) 
    # for the given dates in format "str" inclusive of these dates
    
    # Indicators names
    all_indicator_names = feature_indicator_names + [target_indicator_name]
    print('Selected indicator names:', all_indicator_names)
        
    # Information about stations
    pd.set_option('max_colwidth',200)
    all_id_stations = id_feature_station + [id_target_station]
    data_about = pd.read_csv('../input/wq-southern-bug-river-01052021/PB_stations.csv', sep=';', header=0, encoding='cp1251')
    print('All stations:')
    display(data_about.sort_values(by=['length'], ascending=False))
    print('\nSelected stations:')
    display(data_about[data_about['id'].isin(all_id_stations)])
    
    # Get data from given id_stations with given indicators and dates
    data = pd.read_csv('../input/wq-southern-bug-river-01052021/PB_All_2000_2021.csv', sep=';', header=0)
    data['ds'] = pd.to_datetime(data['date'])
    
    # Data sampling only for selected stations
    df_indicator = data[['id', 'ds'] + all_indicator_names]
    df_indicator = df_indicator[df_indicator['id'].isin(all_id_stations)].dropna().reset_index(drop=True)

    # Set new cols
    cols = []
    for station in all_id_stations:
        for feature in all_indicator_names:
            cols.append(str(station) + "_" + feature)

    df = pd.pivot_table(df_indicator, index=["ds"], columns=["id"], values=all_indicator_names).dropna()
    df.columns = cols
    df = df.reset_index(drop=False)
        
    # Set new target name
    new_target_name = str(id_target_station) + "_" + target_indicator_name
    
    # Get data between given dates
    df = df[(df['ds']>=date_start) & (df['ds']<=date_end)].reset_index(drop=True)
    
    return df, all_indicator_names, all_id_stations, new_target_name

In [ ]:
df, all_indicator_names, all_id_stations, target_name = get_water_data(target_indicator_name, 
                                                                           id_target_station,
                                                                           feature_indicator_names = feature_indicator_names,
                                                                           id_feature_station=id_feature_station)
print(f'\nData for processing (target name - "{target_name}"):')
df

In [ ]:
def get_tsfresh_features(data):
    # Get statistic features using library TSFRESH 
    # Thanks to https://www.kaggle.com/code/vbmokin/btc-growth-forecasting-with-advanced-fe-for-ohlc
    
    # Extract features
    extracted_features = extract_features(data, column_id="ds", column_sort="ds")
    
    # Drop features with NaN
    extracted_features_clean = extracted_features.dropna(axis=1, how='all').reset_index(drop=True)
    
    # Drop features with constants
    cols_std_zero  = []
    for col in extracted_features_clean.columns:
        if extracted_features_clean[col].std()==0:
            cols_std_zero.append(col)
    extracted_features_clean = extracted_features_clean.drop(columns = cols_std_zero)

    extracted_features_clean['ds'] = data['ds']   # For the merging
    
    return extracted_features_clean

In [ ]:
%%time
# FE with TSFRESH
extracted_features_clean = get_tsfresh_features(df[['ds', target_name]])
extracted_features_clean

In [ ]:
extracted_features_clean.describe()

In [ ]:
# Extracted features by TSFRESH with cleaning
extracted_features_clean.columns.tolist()

In [ ]:
# Get all features
df = pd.merge(df, extracted_features_clean, how='left', on='ds')
df

In [ ]:
df.shape

In [ ]:
# Synthesis dataframe with anomalous dates for Facebook Prophet
if is_anomalies:
    anomalous_dates = ['2014-02-09', '2013-04-04']
    holidays_df = pd.DataFrame(columns = ['ds', 'lower_window', 'upper_window', 'prior_scale'])
    holidays_df['ds'] = anomalous_dates
    holidays_df['holiday'] = 'anomalous_dates'
    holidays_df['lower_window'] = 0
    holidays_df['upper_window'] = 0
    holidays_df['prior_scale'] = 10
    display(holidays_df)

In [ ]:
def cut_data(df, y, num_start, num_end):
    # Cutting dataframe df and array or list for [num_start, num_end-1]        
    df2 = df[num_start:(num_end+1)]
    y2 = y[num_start:(num_end+1)] if y is not None else None
    return df2, y2

In [ ]:
def get_target_mf(df, forecasting_days, col=target_name):
    # Get target as difference of the df[col] 
    # Returns target which is shifted for forecasting_days days in the dataframe df
    # "target_name" -> "target" 
    df['target'] = df[target_name].shift(-forecasting_days)
    
    return df

In [ ]:
def get_train_valid_test_ts(df, forecasting_days, target=target_name):
    # Get training, validation and test datasets with target for Time Series models
    
    # Data prepairing
    df = df.dropna(how="any").reset_index(drop=True)
    df = df[['ds', target_name]]
    df.columns = ['ds', 'y']        
    y = None

#     
    
    N = len(df)
    train, _ = cut_data(df, y, 0, N-2*forecasting_days-1)
    valid, _ = cut_data(df, y, N-2*forecasting_days, N-forecasting_days-1)
    test, _ = cut_data(df, y, N-forecasting_days, N)
    
    # Train+valid - for optimal model training
    train_valid = pd.concat([train, valid])

    print(f'Origin dataset has {len(df)} rows and {len(df.columns)} features')
    print(f'Get training dataset with {len(train)} rows')
    print(f'Get validation dataset with {len(valid)} rows')
    print(f'Get test dataset with {len(test)} rows')
    
    return train, valid, test, train_valid

In [ ]:
def calc_metrics(type_score, list_true, list_pred):
    # Calculation score with type=type_score for list_true and list_pred 
    if type_score=='r2_score':
        score = r2_score(list_true, list_pred)
    elif type_score=='rmse':
        score = mean_squared_error(list_true, list_pred, squared=False)
    elif type_score=='mape':
        score = mean_absolute_percentage_error(list_true, list_pred)
    return score

In [ ]:
def result_add_metrics(result, n, y_true, y_pred):
    # Calculation and addition metrics into dataframe result[n,:]
    
    result.loc[n,'r2_score'] = calc_metrics('r2_score', y_true, y_pred)
    result.loc[n,'rmse'] = calc_metrics('rmse', y_true, y_pred)      # in coins
    result.loc[n,'mape'] = 100*calc_metrics('mape', y_true, y_pred)  # in %
    
    return result

In [ ]:
# Results of all models
result = pd.DataFrame(columns = ['name_model', 'type_data', 'r2_score', 'rmse', 'mape', 'params', 'ypred'])

In [ ]:
# Get datasets
if is_Prophet:
    train_ts, valid_ts, test_ts, train_valid_ts = get_train_valid_test_ts(df.copy(), forecasting_days, target=target_name)
    
    if not is_anomalies:
        holidays_df = None

In [ ]:
def prophet_modeling(result, 
                     indicator, 
                     train, 
                     test, 
                     holidays_df, 
                     period_days,
                     fourier_order_seasonality,
                     forecasting_period,
                     name_model,
                     type_data):
    # Performs FB Prophet model training for given train dataset, holidays_df and seasonality_mode
    # Performs forecasting with period by this model, visualization and error estimation
    # df - dataframe with real data in the forecasting_period
    # can be such combinations of parameters: train=train, test=valid or train=train_valid, test=test
    # Save results into dataframe result
    
    # Build Prophet model with parameters and structure 
    model = Prophet(daily_seasonality=False, 
                    weekly_seasonality=False, 
                    yearly_seasonality=False, 
                    changepoint_range=1, 
                    changepoint_prior_scale = 0.5, 
                    holidays=holidays_df, 
                    seasonality_mode = 'multiplicative'
                   )
    model.add_seasonality(name='seasonality', period=period_days, 
                          fourier_order=fourier_order_seasonality, 
                          mode = 'multiplicative', prior_scale = 0.5)
    # Training model for df
    model.fit(train)
    
    # Make a forecast
    future = model.make_future_dataframe(periods = forecasting_period)
    forecast = model.predict(future)
    
    # Draw plot of the values with forecasting data
    figure = model.plot(forecast, xlabel = 'ds', ylabel = f"{name_model} for {indicator}")
    
    # Draw plot with the components (trend and seasonalities) of the forecasts
    figure_component = model.plot_components(forecast)
    
    # Ouput the prediction for the next time on forecasted_days
    #forecast[['yhat_lower', 'yhat', 'yhat_upper']] = forecast[['yhat_lower', 'yhat', 'yhat_upper']].round(1)
    #forecast[['ds', 'yhat_lower', 'yhat', 'yhat_upper']].tail(forecasting_period)
    
    # Forecasting data by the model
    ypred = forecast['yhat'][-forecasting_period:]
    #print(ypred)
    # Save results
    n = len(result)
    result.loc[n,'name_model'] = f"Prophet_{name_model}"
    result.loc[n,'type_data'] = type_data
    result.at[n,'params'] = [period_days]+[fourier_order_seasonality]
    result.at[n,'ypred'] = ypred
    #result = result_add_metrics(result, n, test['y'], y_pred)
    
    return result, ypred

In [ ]:
%%time
# Models tuning
if is_Prophet:
    for period_days in [2, 3, 5, 10]:
        for fourier_order_seasonality in [3, 12]:
            result, _ = prophet_modeling(result, 
                                         target_indicator_name, 
                                         train_ts, 
                                         valid_ts, 
                                         holidays_df, 
                                         period_days,
                                         fourier_order_seasonality,
                                         forecasting_days,
                                         f'{period_days}_days_{fourier_order_seasonality}_order',
                                         'valid')

In [ ]:
# Get datasets
if is_ARIMA:
    train_ts, valid_ts, test_ts, train_valid_ts = get_train_valid_test_ts(df.copy(), forecasting_days, target=target_name)

In [ ]:
def acf_pacf_draw(df, lag_num=40, acf=True, pacf=True, title="", ylim=1):
    # Draw plots named title with ACF and PACF for dataframe df
    
    num_plots = 1+int(acf)+int(pacf)
    fig, ax = plt.subplots(1,num_plots,figsize=(12,6))
    # 'Original Series'
    ax[0].plot(df.values.squeeze())
    
    if acf:
        # ACF drawing
        plot_acf(df.values.squeeze(), lags=lag_num, ax=ax[1])
        ax[1].set(ylim=(-ylim, ylim))
        
        if pacf:
            # PACF drawing
            plot_pacf(df.values.squeeze(), lags=lag_num, ax=ax[2])
            ax[2].set(ylim=(-ylim, ylim))
        
    elif pacf:
        # PACF drawing
        plot_pacf(df.values.squeeze(), lags=lag_num, ax=ax[1])
        ax[1].set(ylim=(-ylim, ylim))

    fig.suptitle(title)
    plt.show()

In [ ]:
def arima_fit(df, col, order=(1,1,1)):
    # ARIMA model fitting for series df[col]
    
    model = sm.tsa.arima.ARIMA(df[col].values.squeeze(), order=order)
    model = model.fit()
    return model

In [ ]:
def get_residual_errors(model):
    # Calculation and drawing the plot residual errors for ARIMA model
    residuals = pd.DataFrame(model.resid)
    fig, ax = plt.subplots(1,2, figsize=(12,6))
    residuals.plot(title="Residuals", ax=ax[0])
    residuals.plot(kind='kde', title='Density', ax=ax[1])
    plt.show()

In [ ]:
def arima_forecasting(result, model, params, name_model, df, type_data):
    # Data df (validation or test) forecasting on the num days by the model 
    # with params and save metrics to result 
    
    ypred = model.forecast(steps=len(df))
    
    n = len(result)
    result.loc[n,'name_model'] = name_model
    result.loc[n,'type_data'] = type_data
    result.at[n,'params'] = params
    result.at[n,'ypred'] = ypred
    #result = result_add_metrics(result, n, df['y'], y_pred)
    
    return result

In [ ]:
%%time
if is_ARIMA:
    # Automatic tuning of the ARIMA model 
    model_auto = pm.auto_arima(train_ts['y'].values, 
                               start_p=4,        # start p
                               start_q=4,        # start q
                               test='adf',       # use adftest to find optimal 'd'
                               max_p=5, max_q=5, # maximum p and q
                               m=1,              # frequency of series (1 - No Seasonality)
                               d=None,           # let model determine 'd'
                               seasonal=False,   # No Seasonality
                               start_P=0,        
                               D=0, 
                               start_Q=0,
                               trace=True,
                               error_action='ignore',  
                               suppress_warnings=False, 
                               stepwise=True     # use the stepwise algorithm outlined in Hyndman and Khandakar (2008) 
                                                 # to identify the optimal model parameters. 
                                                 # The stepwise algorithm can be significantly faster than fitting all 
                                                 # hyper-parameter combinations and is less likely to over-fit the model
                              )

    print(model_auto.summary())

In [ ]:
if is_ARIMA:
    # Get orders of the best model from AutoARIMA
    arima_orders_best = list(model_auto.get_params().get('order'))
    print(f"Optimal parameters are {arima_orders_best}")
    model_auto = arima_fit(train_ts, 'y', order=(arima_orders_best[0],arima_orders_best[1],arima_orders_best[2]))

In [ ]:
if is_ARIMA:
    # Best model from AutoARIMA
    fig = model_auto.plot_diagnostics(figsize=(12,10))
    plt.show()

In [ ]:
df

In [ ]:
# Get datasets
if is_other_ML:
    df2 = get_target_mf(df, forecasting_days, col=target_name)
    train_mf, ytrain_mf, valid_mf, yvalid_mf, test_mf, ytest_mf, train_valid_mf, y_train_valid_mf, starting_point = \
                                    get_train_valid_test_mf(df2.copy(), forecasting_days, target='target')

In [ ]:
if is_other_ML:
    # Set parameters of models
    models = pd.DataFrame(columns = ['name', 'model', 'param_grid'])

    # Linear Regression
    n = len(models)
    models.loc[n, 'name'] = 'Linear Regression'
    models.at[n, 'model'] = LinearRegression()
    models.at[n, 'param_grid'] = {'fit_intercept' : [True, False]}

                                   
    # Support Vector Machines
    n = len(models)
    models.loc[n, 'name'] = 'Support Vector Machines'
    models.at[n, 'model'] = SVR()
    models.at[n, 'param_grid'] = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                                  'C': np.linspace(1, 15, 15),
                                  'tol': [1e-3, 1e-4]
                                 }

        # Random Forest Classifier
    n = len(models)
    models.loc[n, 'name'] = 'Random Forest Regressor'
    models.at[n, 'model'] = RandomForestRegressor()
    models.at[n, 'param_grid'] = {'n_estimators': [40, 50, 60, 80], 
                                  'min_samples_split': [30, 40, 50, 60], 
                                  'min_samples_leaf': [10, 12, 15, 20, 50],
                                  'max_features': ['auto'], 
                                  'max_depth': [3, 4, 5, 6]                   
                                 }

    # Bagging Classifier
    n = len(models)
    models.loc[n, 'name'] = 'Bagging Regressor'
    models.at[n, 'model'] = BaggingRegressor()
    models.at[n, 'param_grid'] = {'max_features': np.linspace(0.05, 0.8, 1),
                                  'n_estimators': [3, 4, 5, 6],
                                  'warm_start' : [False]
                                 }

    # XGB Classifier
    n = len(models)
    models.loc[n, 'name'] = 'XGB Regressor'
    models.at[n, 'model'] = xgb.XGBRegressor()
    models.at[n, 'param_grid'] = {'n_estimators': [50, 70, 90], 
                                  'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                  'max_depth': [3, 4, 5]
                                 }

    
models

In [ ]:
def model_prediction(result, models, train_features, valid_features, train_labels, valid_labels):    
    # Models training and data prediction for all models from DataFrame models
    # Saving results for validation dataset into dataframe result
    
    def calc_add_score(res, n, type_score, list_true, list_pred, feature_end):
        # Calculation score with type=type_score for list_true and list_pred 
        # Adding score into res.loc[n,...]
        res.loc[i, type_score + feature_end] = calc_metrics(type_score, list_true, list_pred)
        return res
    
    # Results
    model_all = []

    for i in range(len(models)):
        # Training
        print(f"Tuning model '{models.loc[i, 'name']}'")
        model = GridSearchCV(models.at[i, 'model'], models.at[i, 'param_grid'])
        model.fit(train_features, train_labels)
        model_all.append(model)
        print(f"Best parameters: {model.best_params_}\n")
        
        # Prediction
        ypred = model.predict(valid_features)
        
        # Scoring and saving results into the main dataframe result
        n = len(result)
        result.loc[n,'name_model'] = f"{models.loc[i, 'name']}"
        result.loc[n,'type_data'] = "valid"
        result.at[n,'params'] = model.best_params_
        result.at[n,'ypred'] = ypred
        #result = result_add_metrics(result, n, valid_labels, valid_pred)
        
    return result, model_all

In [ ]:
%%time
if is_other_ML:
    # Models tuning and the forecasting
    result, model_all = model_prediction(result, models, train_mf, valid_mf, ytrain_mf, yvalid_mf)

In [ ]:
def recovery_prediction(y, starting_point):
    # Recovering prediction of multi-factors model for shifted col to col in the dataframe df
    # y has type np.array
    # starting_point is dictionary with start values for the recovering data
    # Returns y (np.array) with recovering data
    
    return np.insert(y, 0, starting_point).cumsum()[1:]

In [ ]:
def result_recover_and_metrics(result, df_ts, type_data, start_points):
    # Recovering prediction: from shifted_Close to Close
    # Calculation metrics for recovering ypred forecasting for all models in result
    # ypred real is from df_ts['y']
    # start points value for the recovering is from dictionary start_points
    # type_data = 'valid' or 'test'

    for i in range(len(result)):
        if (result.loc[i, 'type_data']==type_data) and (result.loc[i, 'mape'] is np.nan):
            ypred = result.loc[i, 'ypred']

            # Recovering ypred for multi-factors models
            if not (str(result.loc[i, 'type_model']) in ['Prophet', 'ARIMA']):
                # Multi-factors model
                # Get start points value for the recovering
                start_point_value = start_points['valid_start_point'] if type_data=='valid' else start_points['test_start_point']
                # Recovering prediction
                ypred = recovery_prediction(ypred, start_point_value)            

            # Calculation metrics
            result = result_add_metrics(result, i, df_ts['y'], ypred)
    
    return result

In [ ]:
# Dispay and save all results for validation dataset
if len(result) > 0:
    
    # Get type of each model
    result['type_model'] = result['name_model'].str.split('_').str[0]

    # Calculation metrics for recovering prediction ypred for validation dataset by all models 
    result = result_recover_and_metrics(result, valid_ts, 'valid', starting_point)
    display(result[['name_model', 'type_data', 'r2_score', 'rmse', 'mape']].sort_values(by=['type_data', 'mape', 'rmse'], ascending=True))
    
    # Save results
    num_models = len(result[result['type_data']=='valid']['name_model'].unique().tolist())
    print(f"Number of models built - {num_models}")
    result.to_csv(f'result_of_{num_models}_models_for_forecasting_days_{forecasting_days}.csv')
else: 
    print('There are no tuned models!')

In [ ]:
def get_model_opt(name_model, params):
    # Model tuning for the name_model
    
    print(name_model)
    if name_model=='Linear Regression':
        model = LinearRegression(**params)
        
              
    elif name_model=='Support Vector Machines':
        model = SVR(**params)
        
            
    elif name_model=='Random Forest Regressor':
        model = RandomForestRegressor(**params)
        
    elif name_model=='Bagging Regressor':
        model = BaggingRegressor(**params)
    
            
    elif name_model=='XGB Regressor':
        model = xgb.XGBRegressor(**params)
        
    else: model = None
        
    return model

In [ ]:
def get_params_optimal_model(result, main_metrics):
    # Get parameters of the optimal model from dataframe result by main_metrics

    # Set the data type to float (just in case)
    result[main_metrics] = result[main_metrics].astype('float')
    
    # Choose the optimal model
    opt_result = result[result['type_data']=='valid'].reset_index(drop=True)
    if main_metrics=='r2_score':
        opt_model = opt_result.nlargest(1, main_metrics)
    else:
        # 'mape' or 'rmse'
        opt_model = opt_result.nsmallest(1, main_metrics)
    display(opt_model[['name_model', 'r2_score', 'rmse', 'mape', 'params']])

    # Get parameters of the optimal model
    opt_name_model = opt_model['name_model'].tolist()[0]
    opt_type_model = opt_model['type_model'].tolist()[0]
    opt_params_model = opt_model['params'].tolist()[0]
    print(f'Optimal model by metrics "{main_metrics}" is "{opt_name_model}" with type "{opt_type_model}" parameters {opt_params_model}')
    
    return opt_name_model, opt_type_model, opt_params_model

In [ ]:
def model_training_forecasting(result, df, y, test, ytest,  
                               name_model, type_model, params, type_test='1'):
    # Model training for df and y
    # Forecasting ypred
    # type_model = 'Prophet' or "ARIMA" or 'Other ML'
    # type_test = '1' (with find optimal parameters by GridSearchCV) 
    # type_test = '2' (with optimal parameters - without GridSearchCV)
    # return params and metrics in the dataframe result
    
    if type_model=='Prophet':    
        season_days_optimal = params[0]
        fourier_order_seasonality_optimal = params[1]
        model_opt = None
        _, ypred = prophet_modeling(result, 
                                    target_indicator_name, 
                                    df, 
                                    test, 
                                    holidays_df, 
                                    season_days_optimal,
                                    fourier_order_seasonality_optimal,
                                    forecasting_days,
                                    f'{type_model}_optimal',
                                    'test')        
    elif type_model=='ARIMA':
        season_days_optimal = params[0]
        fourier_order_seasonality_optimal = params[1]        
        model_opt = None
        
        # Training ARIMA optimal model for training+valid dataset
        df['y'] = y
        model_opt = arima_fit(df, 'y', order=(params[0],params[1],params[2]))        

        # Model diagnostics
        fig = model_opt.plot_diagnostics(figsize=(12,10))
        plt.show()

        # Plot residual errors
        get_residual_errors(model_opt)

        # Test forecasting and save result
        ypred = model_opt.forecast(steps=len(test))        

    else:
        # Other ML model
        # Training ML optimal model for training+valid dataset
        print(f"Tuning model '{name_model}'")
        models_opt_number = models[models['name']==name_model].index.tolist()[0]
        #print(f"Model - {models.at[models_opt_number,'model']} with parameters {params}")
        if type_test=='1':
            model_opt = GridSearchCV(models.at[models_opt_number,'model'], models.at[models_opt_number,'param_grid'])
        else:
            # type_test=='2'
            model_opt = get_model_opt(models.at[models_opt_number,'name'], params)
        model_opt.fit(df, y)
        
        # Forecasting
        ypred = model_opt.predict(test)

        
    # Scoring and saving results into the dataframe result
    n = len(result)-1
    result.loc[n,'name_model'] = f"{type_model}_optimal"
    result.loc[n,'type_data'] = "test"
    result.loc[n,'type_model'] = type_model
    result.at[n,'params'] = params
    result.at[n,'ypred'] = ypred
    #result = result_add_metrics(result, n, ytest, ypred)
    
    return result, model_opt, ypred

In [ ]:
def get_optimal_model_and_forecasting(result, main_metrics, start_points):
    # Choosion the optimal model from dataframe result by main_metrics
    # Tuning optimal model for big dataset train+valid 
    # Test forecasting and drawing it
    # Returns the optimal model and it's name

    
    if len(result) > 0:
        # Get parameters of the optimal model from dataframe result by main_metrics
        opt_name_model, opt_type_model, opt_params_model = get_params_optimal_model(result, 
                                                                                    main_metrics)
        # Set datasets for the final tuning and testing by optimal model
        if (opt_type_model=='Prophet') or (opt_type_model=='ARIMA'):
            train_valid = train_valid_ts.copy()
            y_train_valid = train_valid_ts['y'].copy()
            test = test_ts.copy()
            ytest = test_ts['y'].copy()
            
        else:
            # Multi-factors ML models
            train_valid = train_valid_mf.copy()
            y_train_valid = y_train_valid_mf.copy()
            test = test_mf.copy()
            ytest = ytest_mf.copy()
    
        # Optimal model training for train+valid and test forecasting
        result, model_opt, ypred = model_training_forecasting(result, train_valid, y_train_valid,
                                                              test, ytest,
                                                              opt_name_model, opt_type_model, 
                                                              opt_params_model, '1')
        
        # Calculation metrics for recovering prediction ypred for test dataset by the optimal model
        result = result_recover_and_metrics(result, test_ts, 'test', start_points)
        
        # Drawing plot for prediction for the test data 
        if not ((opt_type_model=='Prophet') or (opt_type_model=='ARIMA')):
            # Recovery values target_name
            ytest_plot = recovery_prediction(ytest.values, start_points['test_start_point'])
            ypred_plot = recovery_prediction(ypred, start_points['test_start_point'])
        else:
            ytest_plot = ytest.copy()
            ypred_plot = ypred.copy()
            
        # Drawing 
        plt.figure(figsize=(12,8))
        x = np.arange(len(ytest_plot))
        plt.scatter(x, ytest_plot, label = "Target test data", color = 'g', s=100)
        plt.scatter(x, ypred_plot, label = f"{opt_name_model} forecasting", color = 'r', s=50)
        plt.title(f'Forecasting of test data using the "{opt_name_model}" model, which is optimal for "{main_metrics}" metrics')
        plt.ylim(0)
        plt.legend(loc='lower right')
        plt.grid(True)
        
        return opt_name_model

In [ ]:
# Get the optimal model by different metrics
if len(result) > 0:
    for valid_metrics in ['r2_score', 'rmse', 'mape']:
        get_optimal_model_and_forecasting(result, valid_metrics, starting_point)    

In [ ]:
# Training ML optimal model for training+valid dataset
# Get parameters of the optimal model from dataframe result (without Time Series models) by main_metrics
if is_other_ML:
    main_metrics = 'r2_score'
    if (len(result) > 0) and (len(models) > 0):
        result_nonTS = result[(result['type_model']!='Prophet') & (result['type_model']!='ARIMA')].reset_index(drop=True)
        opt_name_model2, opt_type_model2, opt_params_model2 = get_params_optimal_model(result_nonTS, 
                                                                                main_metrics)

        result, model_opt, ypred = model_training_forecasting(result, 
                                                              train_valid_mf, 
                                                              y_train_valid_mf,
                                                              test_mf, 
                                                              ytest_mf,
                                                              opt_name_model2, 
                                                              opt_type_model2, 
                                                              opt_params_model2, 
                                                              '2')

In [ ]:
# All features names
if is_other_ML:
    coeff = pd.DataFrame(train_valid_mf.columns)
    coeff.columns = ['feature']

In [ ]:
def add_fi_coeff(coeff, col, list_new_fi_coeff=None, df_new_fi_coeff=None):
    # Adds new importance of features as feature col
    # from list list_new_fi_coeff or dataframe df_new_fi_coeff
    # to the resulting dataframe coeff with feature names 
    # Missed importance values are replaced by zero
    
    if list_new_fi_coeff is not None:
        df_new_fi_coeff = coeff[['feature']].copy()
        df_new_fi_coeff["score"] = pd.Series(list_new_fi_coeff)
    
    if df_new_fi_coeff is not None:
        # Rename df_new_fi_coeff
        df_new_fi_coeff.columns = ['feature', 'score']   # to the plot drawing
        df_new_fi_coeff[col] = df_new_fi_coeff['score']  # to the merging and saving
        
        # Merging dataframes - coeff of all features with new_fi_coeff
        coeff = coeff.merge(df_new_fi_coeff[['feature', col]], on='feature', how='left').fillna(0)
        
        is_score = True
    else:
        print(f'Data is absent fol {col}')
        is_score = False
        coeff = None
    
    return coeff, df_new_fi_coeff, is_score

In [ ]:
# Feature importance diagram with SHAP
if is_other_ML:
    if (len(result) > 0) and (len(models) > 0):
        print('Feature importance diagram with SHAP:')
        try:
            # Trees
            explainer = shap.TreeExplainer(model_opt)
            shap_values = explainer.shap_values(test_mf)
            shap.summary_plot(shap_values, test_mf, plot_type="bar", feature_names=coeff['feature'].tolist())
            shap.summary_plot(shap_values, test_mf)

            # Save permutation feature importance values
            coeff, _, is_SHAP_successfully = add_fi_coeff(coeff, 'shap_fi_score', shap_values)
        except: 
            try:
                # Other types of models
                explainer = shap.KernelExplainer(model_opt.predict, train_valid_mf)
                shap_values = explainer.shap_values(test_mf)

                # Plot drawing
                shap.summary_plot(shap_values, test_mf, plot_type="bar", feature_names=coeff['feature'].tolist())
                shap.summary_plot(shap_values, test_mf)

                # Get feature importance values from shap_values format
                # Thanks to https://stackoverflow.com/a/69523421/12301574
                shap_values_all = pd.DataFrame(shap_values, columns = test_mf.columns)
                vals = np.abs(shap_values_all.values).mean(0)
                shap_importance = pd.DataFrame(list(zip(test_mf.columns, vals)),
                                                  columns=['feature','score'])            

                # Saving feature importance values
                coeff, _, is_SHAP_successfully = add_fi_coeff(coeff, 'shap_fi_score', None, shap_importance)            

            except: 
                is_SHAP_successfully = False

        if not is_SHAP_successfully:
            print('Feature importance diagram for this optimal model is not supported in SHAP')

In [ ]:
# Force plot - Feature importance diagram with SHAP for the certaion row in test_mf
if is_other_ML:
    if (len(result) > 0) and (len(models) > 0):
        row_number_in_test_mf = 0
        print('Feature importance diagram as the Force plot with SHAP:')
        if is_SHAP_successfully:
            shap.initjs()
            shap.force_plot(explainer.expected_value, shap_values[0,:], 
                            test_mf.loc[test_mf.index.tolist()[row_number_in_test_mf],:],
                            feature_names=coeff['feature'].tolist(),
                            matplotlib=True, show=False)
            plt.savefig('force_plot.png')

In [ ]:
# Creation and drawing the feature importance diagrams
if is_other_ML:
    if (len(result) > 0) and (len(models) > 0):

        # Coefficients
        if opt_name_model2=='XGB Regressor':
            print('Feature importance diagram')
            # Coef. of the feature with nonzero importance
            xgb_coeff = pd.DataFrame.from_dict(model_opt.get_booster().get_score(importance_type='weight'), orient='index').reset_index(drop=False)
            coeff, _, is_score = add_fi_coeff(coeff, 'xgb_fi_coeff', None, xgb_coeff)

            # With the library xgboost
            fig =  plt.figure(figsize = (15,15))
            axes = fig.add_subplot(111)
            xgb.plot_importance(model_opt,ax = axes,height = 0.5)
            plt.show()
            plt.close()

        else:
            # With the library sklearn
            try:
                coef_model = model_opt.coef_
                coeff, coeff_new, is_score = add_fi_coeff(coeff, 'lr_fi_score', coef_model)
            except:
                try:
                    coef_model = feature_importances_
                    coeff, coeff_new, is_score = add_fi_coeff(coeff, 'model_fi_score', coef_model)
                except: 
                    print('The importance of the feature could not be obtained')
                    is_score = False

            if is_score:
                # Plot drawing
                coeff_non_zero = coeff_new[coeff_new['score']>0]
                plt.figure(figsize=(12, int(len(coeff_non_zero)*0.4)))
                coeff_non_zero = coeff_non_zero.sort_values(by='score', ascending=True)
                plt.barh(coeff_non_zero["feature"], coeff_non_zero["score"])
                plt.title("Feature importance diagram")
                plt.axvline(x=0, color=".5")
                plt.xlabel("Coefficient values")
                plt.subplots_adjust(left=0.3)

In [ ]:
# Permutation feature importance diagram
if is_other_ML:
    if (len(result) > 0) and (len(models) > 0):
        try:
            perm_importance = permutation_importance(model_opt, test_mf, ytest_mf)

            # Save permutation feature importance values
            coef_model = perm_importance.importances_mean
            coeff, coeff_new, is_score = add_fi_coeff(coeff, 'perm_fi_score', coef_model)

            print('Permutation feature importance diagram:') 
            coeff_non_zero = coeff_new[coeff_new['score'].abs()>1e-4]
            coeff_non_zero = coeff_non_zero.sort_values(by='score', ascending=True)
            plt.figure(figsize=(12, int(len(coeff_non_zero)*0.4)))
            plt.barh(coeff_non_zero["feature"], coeff_non_zero["score"])
            plt.xlabel("Permutation Importance")
            plt.show()
            is_perm_importance = True
        except: print('Permutation feature importance diagram for this optimal model is not supported')

In [ ]:
# Display and saving features importance values
if is_other_ML:
    if coeff.isna().sum().sum()==0:
        print('Feature importance values:')
        fi_cols = coeff.columns.tolist()[1:]
        if len(fi_cols) > 0:
            coeff = coeff.sort_values(by=fi_cols, ascending=False)
            display(coeff)
        coeff.to_csv(f'feature_importance_for_optimal_model_{opt_name_model2}.csv', index=False)